In [55]:
!pip install tensorflow 

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 497.5 MB 12 kB/s              
     |████████████████████████████████| 14.5 MB 26.3 MB/s            
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     |████████████████████████████████| 462 kB 50.3 MB/s            
  Using cached tensorboard-2.8.0-py3-none-any.whl (5.8 MB)
     |████████████████████████████████| 2.1 MB 47.8 MB/s            
     |████████████████████████████████| 4.3 MB 53.2 MB/s            
  Using cached termcolor-1.1.0-py3-none-any.whl
     |████████████████████████████████| 1.4 MB 54.0 MB/s            
  Using

In [7]:
%%bash
rm -r input
rm -r output
mkdir input
mkdir output

In [1]:
import boto3
import io
import os 
from PIL import Image

from keras.preprocessing.image import ImageDataGenerator
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

from sagemaker import get_execution_role

In [ ]:
role = get_execution_role()

my_bucket = 'london-innovation-hub'
orig_file = 'xray/feb_dataset/Defect/2021-01-01 07-02-46 - recRTBJBHEM32AK1 - 8610277217_Az183.00_NCT2.tif'
dest_file = './input/file.tif'

client = boto3.client('s3')
paginator = client.get_paginator('list_objects_v2')
result = paginator.paginate(Bucket=my_bucket, Prefix='xray/feb_dataset/DefectSmall', StartAfter='2018')

s3 = boto3.resource('s3')
# Iterate over all Images in Folder 

# create image data augmentation generator
datagen = ImageDataGenerator(height_shift_range=0.5, fill_mode='wrap')

for page in result:
    if "Contents" in page:
        
        count = 0
        length = len(page[ "Contents" ])
        
        for key in page[ "Contents" ]:
            keyString = key[ "Key" ]
            if keyString.split('.')[-1] == 'tif':
                # we now can interate over all strings of objects in the folder
                
                #file name is extracted without extension or path 
                file_name = os.path.splitext(keyString)[0].split("/")[-1]
                #downlaod to sagemaker console
                s3.Bucket(my_bucket).download_file(keyString, f"./input/{file_name}.tif")
                
                #convert image to JPEG
                layer_count = 0 
                im = Image.open(f"./input/{file_name}.tif")
                while True:
                    try:
                        im.seek(layer_count)
                        out = im.convert("RGB")
                        out.save( "./output/" + file_name + ".jpeg", "JPEG", quality=100)
                    except EOFError:
                        break
                    layer_count += 1
                
                # delete .tif in input folder
                for x in os.listdir('./input/'):
                    if x.split('.')[-1] == 'tif':
                        os.remove(f"input/{x}")
                
                # Generate Image Shifts
                img = load_img(f"./output/{file_name}.jpeg")
                data = img_to_array(img)
                samples = expand_dims(data, 0)
                # prepare iterator
                it = datagen.flow(samples, batch_size=1)
                
                # generate samples 
                for i in range(10):
                    # generate batch of images
                    batch = it.next()
                    # convert to unsigned integers for viewing
                    image = batch[0].astype('uint8')
                    im1 = Image.fromarray(image)
                    im1 = im1.save(f"./output/{file_name}_Augmented_VS_{i}.jpeg")
                    
                #upload contents of the Output folder to s3 an ddelete
                for x in os.listdir('./output/'):
                    if x.split('.')[-1] == 'jpeg':
                        with open(f'./output/{x}', 'rb') as data:
                            client.upload_fileobj(data, my_bucket, f'xray/feb_dataset/Defect_JPEG/{x}')
                        os.remove(f"./output/{x}")

                count += 1
                print(f"Completed {(count/length) * 100}%", end='\r')